# Brain Tumor Classification
#### Dataset Source : https://www.kaggle.com/navoneel/brain-mri-images-for-brain-tumor-detection

In [1]:
!pip install gdown
!gdown --id 1080ExOfZ2byjIGpg7NWo5H5eqog0-k2V

Downloading...
From: https://drive.google.com/uc?id=1080ExOfZ2byjIGpg7NWo5H5eqog0-k2V
To: /content/brain-tumor-dataset.zip
100% 15.8M/15.8M [00:00<00:00, 139MB/s]


## Importing Library

In [10]:
import tensorflow as tf
import os
import zipfile
import shutil

from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Unzipping File

In [3]:
# Targetting zip file
zip_file = 'brain-tumor-dataset.zip'
extracting = zipfile.ZipFile(zip_file, 'r')

# Creating uncleaned dataset directory
os.mkdir('uncleaned_dataset')
uncleaned_dataset_dir = 'uncleaned_dataset'

# Extracting
extracting.extractall(uncleaned_dataset_dir)
extracting.close()

# Remove zip file
os.remove(zip_file)

## Remove Redundant Directory

In [4]:
shutil.rmtree(os.path.join(uncleaned_dataset_dir, 'brain_tumor_dataset'))

## Splitting Dataset

In [5]:
# Count File
print("Number of image in 'No' Class : {} ".format(len(os.listdir(os.path.join(uncleaned_dataset_dir, 'no')))))
print("Number of image in 'Yes' Class : {} ".format(len(os.listdir(os.path.join(uncleaned_dataset_dir, 'yes')))))

Number of image in 'No' Class : 98 
Number of image in 'Yes' Class : 155 


In [6]:
# Decided Training Length
training_length = 80

# Created Cleaned Dataset Directory
os.mkdir('dataset')
dataset_dir = 'dataset'

os.mkdir('dataset/training')
os.mkdir('dataset/validation')

training_dir = 'dataset/training'
validation_dir = 'dataset/validation'

os.mkdir(os.path.join(training_dir, 'yes'))
training_dir_yes = os.path.join(training_dir, 'yes')

os.mkdir(os.path.join(training_dir, 'no'))
training_dir_no = os.path.join(training_dir, 'no')

os.mkdir(os.path.join(validation_dir, 'yes'))
validation_dir_yes = os.path.join(validation_dir, 'yes')

os.mkdir(os.path.join(validation_dir, 'no'))
validation_dir_no = os.path.join(validation_dir, 'no')

In [7]:
# Move 80 image file from uncleaned dataset to dataset/training

# class NO
count = 0
no_uncleaned_path = os.path.join(uncleaned_dataset_dir, 'no')

for img_file in os.listdir(no_uncleaned_path):
  count += 1
  img_path_before = os.path.join(no_uncleaned_path, img_file)
  img_path_after = os.path.join(training_dir_no, img_file)
  
  if count <= training_length:
    shutil.move(img_path_before, img_path_after)

# class YES
count = 0
no_uncleaned_path = os.path.join(uncleaned_dataset_dir, 'yes')

for img_file in os.listdir(no_uncleaned_path):
  count += 1
  img_path_before = os.path.join(no_uncleaned_path, img_file)
  img_path_after = os.path.join(training_dir_yes, img_file)
  
  if count <= training_length:
    shutil.move(img_path_before, img_path_after)

In [8]:
# Move 80 image file from uncleaned dataset to dataset/validation

# class NO
no_uncleaned_path = os.path.join(uncleaned_dataset_dir, 'no')

for img_file in os.listdir(no_uncleaned_path):
  img_path_before = os.path.join(no_uncleaned_path, img_file)
  img_path_after = os.path.join(validation_dir_no, img_file)

  shutil.move(img_path_before, img_path_after)

# class YES
no_uncleaned_path = os.path.join(uncleaned_dataset_dir, 'yes')

for img_file in os.listdir(no_uncleaned_path):
  img_path_before = os.path.join(no_uncleaned_path, img_file)
  img_path_after = os.path.join(validation_dir_yes, img_file)

  shutil.move(img_path_before, img_path_after)

In [9]:
# Remove uncleaned dataset dir
shutil.rmtree(uncleaned_dataset_dir)

## Preprocessing Image

In [11]:
training_rule = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    shear_range = 0.2,
    rotation_range = 0.2,
    zoom_range = 0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    fill_mode = 'nearest'
)

validation_rule = ImageDataGenerator(
    rescale = 1./255
)

In [19]:
training_generator = training_rule.flow_from_directory(
    training_dir,
    target_size = (150, 150),
    class_mode = 'categorical',
    batch_size = 20,
    shuffle = True
)

validation_generator = validation_rule.flow_from_directory(
    validation_dir,
    target_size = (150, 150),
    batch_size = 20,
    class_mode = 'categorical'
)

Found 160 images belonging to 2 classes.
Found 93 images belonging to 2 classes.


## Build Model

In [21]:
model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),

      tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
      tf.keras.layers.MaxPooling2D(2, 2),

      # tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu')
      # tf.keras.layers.MaxPooling2D(2, 2),

      tf.keras.layers.Flatten(),

      tf.keras.layers.Dense(128, activation = 'relu'),
      tf.keras.layers.Dense(64, activation = 'relu'),
      tf.keras.layers.Dense(2, activation = 'softmax')
])

model.summary()

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]
)

history = model.fit(
    training_generator,
    epochs = 50,
    validation_data = validation_generator
)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 41472)             0         
                                                                 
 dense_12 (Dense)            (None, 128)              

## Saving Model

In [24]:
# os.mkdir('model')
saved_model_path = './model/model.h5'
model.save(saved_model_path)